In [5]:
from PIL import Image
import time
%matplotlib inline
import matplotlib.pyplot as plt
from copy import copy

In [56]:
#imgs = [Image.open('../orig_0.jpg'), Image.open('../orig_1.jpg'), Image.open('../orig_2.jpg')]
imgs = [Image.open('../../../testing_images/1_tree/Tree_1.JPG'),
        Image.open('../../../testing_images/1_tree/Tree_2.JPG'),
        Image.open('../../../testing_images/1_tree/Tree_3.JPG')]
# imgs = [Image.open('../../../testing_images/3_Seashore/1.JPG'),
#         Image.open('../../../testing_images/3_Seashore/2.JPG'),
#         Image.open('../../../testing_images/3_Seashore/3.JPG')]
# imgs = [Image.open('../../../testing_images/3_tree2/1.JPG'),
#         Image.open('../../../testing_images/3_tree2/2.JPG'),
#         Image.open('../../../testing_images/3_tree2/3.JPG')]

In [6]:
def bal(im):
    """
    adjust the balance of the mask
    (re-distribute the histogram so that there are more
    extreme blacks and whites)
    like increasing the contrast, but without clipping
    and maintains overall average image brightness
    """
    h = im.histogram()
    ln = range(len(h))
    up = [sum(h[0: i]) for i in ln]
    lo = [sum(h[i:-1]) for i in ln]
    ct = sum(h)
    st = int(strength*255.)

    lut = [i+st*up[i]*lo[i]*(up[i]-lo[i])/ct**3 for i in ln]
    for i in ln:
        if lut[i]<  1:lut[i]=  1
        if lut[i]>255:lut[i]=255
    return im.point(lut)

In [7]:
def get_masks(imgs):
    """
    create a set of masks from a list of images
    (one mask for every adjacent pair of images
    """
    masks = []
    mask_ct = len(imgs)-1
    imgs = [bal(img.convert(mode='L'), strength) for img in imgs]
    for i in range(mask_ct):
        blend_fraction = .5     #1. - (float(i)+.5)/float(mask_ct)
        m = Image.blend(imgs[i],imgs[i+1],blend_fraction)
        masks.append(m)
        #print blend_fraction
    print 'blending using',mask_ct,'masks'
    return masks

In [8]:
def merge(imgs):
    """
    combine a set images into a smaller set by combinding all
    adjacent images
    """
    masks = get_masks(imgs)
    imx = lambda i:Image.composite(imgs[i],imgs[i+1],masks[i])
    return [imx(i) for i in range(len(masks))]

In [9]:
def merge_all(imgs):
    """
    iteratively merge a set of images until only one remains
    """
    while len(imgs)>1:
        imgs = merge(imgs)
    return imgs[0]

In [48]:
strength = 1

def get_hdr(imgs_o, strgth=0, naturalness=0):
    """
    process the hdr image(s)
    strength - a list or a float that defines how strong the hdr
               effect should be
             - a value of zero will combine images by using a
               greyscale image average
             - a value greater than zero will use higher contrast
               versions of those greyscale images
    naturalness- values between zero and one
             - zero will be a very high-contrast image
             - 1.0 will be a very flat image
             - 0.7 to 0.9 tend to give the best results
    """
    print 'getting saturation image'
    global strength
    strength= strgth
    imgs = copy(imgs_o)
    sat_img = merge_all(imgs)

    print 'getting contrast image'
    imgs.reverse()
    con_img = merge_all(imgs)

    fin = final_blend(con_img,sat_img, naturalness)
    
    fin.show()


def final_blend(im1,im2,naturalness):
    """
    combines a saturated image with a contrast image
    and puts them in a dictionary of completed images
    """
    return Image.blend(im1,im2,naturalness)

In [59]:
%%time
get_hdr(imgs,1,1)

getting saturation image
blending using 2 masks
blending using 1 masks
getting contrast image
blending using 2 masks
blending using 1 masks
CPU times: user 1.25 s, sys: 93.3 ms, total: 1.34 s
Wall time: 1.37 s
